# Import stuff here

In [24]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from sklearn.preprocessing import Normalizer
from gensim import corpora, models, matutils
from gensim.models.ldamulticore import LdaMulticore
from scipy.spatial.distance import cdist
import itertools
from copy import deepcopy
from itertools import chain
import scipy

In [25]:
from input_values import (
    TV_SHOW, 
    PRE_PROCESSED_FILE_NAME, 
    LDA_FILE_NAME, 
    OUT_DIR, 
    GAMMA, 
    TOLERANCE, 
    ITERATIONS, 
    NUM_TOPICS,
    GRAPHML_FILE,
    GRAPH_NODE_FILE,
    TOPIC_VEC_FILE,
    TWITTER_RANK_FILE,
    NUM_OF_INFLUENTIAL_NODES
)

In [26]:
NUM_OF_INFLUENTIAL_NODES = 100
threshold_percentile_for_merge = 50
threshold_percentile_for_split = 50

# Import the graph

In [27]:
def remove_isolated_nodes(G):
    print(nx.info(G))
    isolated_nodes = list(nx.isolates(G))
    print('\nIsolated nodes: {}\n'.format(len(isolated_nodes)))
    print('removing isolated nodes...\n')
    G.remove_nodes_from(isolated_nodes)
    print(nx.info(G))
    return G

def load_graph(graph_file = GRAPHML_FILE):

    graph = nx.read_graphml(GRAPHML_FILE)
    print(nx.info(graph))
    
    return graph

# Import dataframes :-
## user_id_df, graph_nodes_df, twitter_rank_df, topic_vec_df

In [28]:
def get_topic_frame(file, graph_node_file=GRAPH_NODE_FILE):
    graph = pd.read_csv(graph_node_file)
    graph = graph.rename(columns = {'Unnamed: 0':'userid'})
    topic = pd.read_csv(file)
    topic = topic.rename(columns = {'Unnamed: 0':'userid'})
    columns = topic.columns
    new = pd.merge(graph,topic,on = 'userid',how = 'left')
    dic = {'0_x':'0_y','1_x':'1_y','2_x':'2_y','3_x':'3_y','4_x':'4_y','5_x':'5_y',
           '6_x':'6_y','6_x':'6_y','7_x':'7_y','8_x':'8_y','9_x':'9_y'}
    for i in dic:
        new[i] = new[i] + new[dic[i]]
    new = new.drop(columns = dic.values())
    new.fillna(0.11111111)
    new.set_index('userid', inplace=True)
    return new

# Community Detection algorithm below

In [29]:
def get_influential_nodes(graph, df, twitter_rank_df, num_topics=NUM_TOPICS, 
                          num_of_influential_nodes=NUM_OF_INFLUENTIAL_NODES):
    topic_rank = twitter_rank_df.values
    topic_rank_sum = np.sum(topic_rank/num_topics, axis=1)
    average_twitter_rank = np.array(topic_rank_sum[:])

    df['avg_twitter_rank'] = average_twitter_rank
    influential_nodes_index = np.argsort(average_twitter_rank, axis=0).reshape(
        len(average_twitter_rank),1)[:NUM_OF_INFLUENTIAL_NODES, :]

    influential_nodes_index = [int(i) for i in influential_nodes_index]
    influential_nodes = df.iloc[influential_nodes_index].index.tolist()
    
    return influential_nodes

In [30]:
# Community detection algorithm here

def step1_assign_initial_communities(graph, df, influential_nodes):
    # Initial community assignment
    influential_nodes = [int(i) for i in influential_nodes]
    communities = {k: [k] for k in influential_nodes}

    # Remaining nodes
    remaining_nodes = set(df.index) - set(influential_nodes)

    # Get the weighted adjacency matrix
    adjacency_matrix = nx.adjacency_matrix(graph, weight='weight').todense()
    adjacency_matrix_df = pd.DataFrame(data=adjacency_matrix, index=df.index, 
                                       columns=df.index)

    # Get nodes weight with influential users
    nodes_weight_with_influential_nodes_df = adjacency_matrix_df[influential_nodes]
    nodes_weight_with_influential_nodes_df['max_weight_with_influencer'] = nodes_weight_with_influential_nodes_df[
        nodes_weight_with_influential_nodes_df > 0].idxmax(axis=1)
    nodes_weight_with_influential_nodes_df['max_weight_with_influencer'].fillna(False, inplace=True)
    # Main algorithm here


    for node in remaining_nodes:
      # add this node to an influencer’s community if this influencer 
      # and this node have the highest edge weight
      influencer_with_max_weight_with_node = nodes_weight_with_influential_nodes_df[
          'max_weight_with_influencer'].loc[node]
      if isinstance(influencer_with_max_weight_with_node, np.int64):
        communities[influencer_with_max_weight_with_node].append(node)
    
    return communities

In [31]:
def step2_split_community(df, communities, threshold_percentile_for_split):

    # Split these initial communities based on topic vectors.  
    # Given a community of m nodes, we can compute the pairwise cosine-distance of 
    # the topical vectors. This will give us m(m-2)/2 distances.  
    # We then remove a node if its cosine distances from all its neighbors 
    # are below a threshold, say, the first quartile of all the m(m-2)/2 distances.


    topic_vectors = df.iloc[:,:-1].values
    cosine_sim = np.array([]).reshape(len(df),0)
    for node in df.index.tolist():
      cosine_sim = np.c_[cosine_sim, 1 - cdist(topic_vectors, np.matrix(df.loc[node])[:,:NUM_TOPICS])]

    cosine_sim_df = pd.DataFrame(data=cosine_sim, index=df.index, columns=df.index)
    mapped_nodes_in_communities = list(itertools.chain(*communities.values()))
    community_cosine_sim_df = cosine_sim_df.loc[mapped_nodes_in_communities][mapped_nodes_in_communities]
    
    
    split_threshold = np.percentile(community_cosine_sim_df.values, threshold_percentile_for_split)
    #splitting here
    updated_communities = deepcopy(communities)
    for seed_node, community in communities.items():
      if len(community) == 1:
        print('Cannot split for community since it has community: {} since it has only one node'.format(community))
      else:
        # split the community based on topic vectors within a community
        for count_nodes, community_node in enumerate(community):
          if community_node == seed_node:
            continue
          is_cos_dist_bigger_than_threshold = list(community_cosine_sim_df.loc[community_node] > split_threshold)
          if False in is_cos_dist_bigger_than_threshold:
            print('Splitting node: {} from community: {}'.format(community_node, updated_communities[seed_node]))
            updated_communities[community_node] = [community_node]
            updated_communities[seed_node].remove(community_node)
    
    return updated_communities, community_cosine_sim_df, mapped_nodes_in_communities


In [32]:
def get_min_cosine_distance(comm1, community_cosine_sim_df, merge_threshold, mapped_nodes_in_communities):
  if not isinstance(comm1, set):
    comm1 = set(comm1)
  remaining_list = list(set(mapped_nodes_in_communities)-comm1)
  #print('*****',comm1)
  min_cosine_dist_from_comm1 = community_cosine_sim_df[community_cosine_sim_df.index.isin(list(comm1))][remaining_list].idxmax(axis=1).values
  #print('*****',min_cosine_dist_from_comm1)
  min_distance_list = []
  for i in zip(comm1, min_cosine_dist_from_comm1):
    min_distance_list.append(community_cosine_sim_df[i[0]][i[1]])
  #print('*****',min_distance_list)
  community_to_merge = None
  if min(min_distance_list) > merge_threshold:
    community_to_merge = min_cosine_dist_from_comm1[min_distance_list.index(min(min_distance_list))]
  return community_to_merge

def step3_merge_communities(updated_communities, community_cosine_sim_df, threshold_percentile_for_merge, 
                            mapped_nodes_in_communities):
    
    merge_threshold = np.percentile(community_cosine_sim_df.values, threshold_percentile_for_split)
    test_communities = deepcopy(updated_communities)
    #print(test_communities)
    for seed_node, community in updated_communities.items():
      if seed_node in test_communities:
        community_to_merge_with_seed = get_min_cosine_distance(community, community_cosine_sim_df, 
                                                               merge_threshold, mapped_nodes_in_communities)
        #print(seed_node, community_to_merge_with_seed)
        if community_to_merge_with_seed:
          seed_node_to_merge = [key for key, value in test_communities.items() if community_to_merge_with_seed in value][0]
          #print(seed_node, test_communities)
          merging_communities = test_communities.pop(seed_node_to_merge, None)
          #print(seed_node, merging_communities)
          if merging_communities:
            test_communities[seed_node].extend(merging_communities)
    
    return test_communities

In [33]:
def make_partitions(df, communities):
    partitions = dict()

    for k, v in communities.items():
      for i in v:
        partitions[int(i)] = int(k) 
        
    for i in set(df.index)-set(partitions.keys()):
        partitions[i] = i
    
    return partitions

## Evaluation

In [34]:
def get_conductance(graph, partitions):
  conductances_list = []
  conductances_keys = []
  for key, coms in partitions.items():
    try:
        conductances_list.append(nx.conductance(graph, coms))
    except ZeroDivisionError:
        conductances_list.append(np.NAN)
        conductances_keys.append(key)
    else:
        conductances_keys.append(key)
  conductance_measures = dict(zip(conductances_keys, conductances_list))
  if not conductance_measures:
        return {}
  return {'all_conductances': conductance_measures, 'min_conductance': min(conductance_measures.values()),
          'max_conductance': max(conductance_measures.values()), 
          'avg_conductance': sum(conductance_measures.values())/len(partitions)}

def get_triangle_participation_ratio(graph, partitions):
  if nx.is_directed(graph):
    graph = nx.to_undirected(graph)
  tpr_measures = dict(zip(partitions.keys(), [triangle_participation_ratio(graph, coms) 
                                              for coms in partitions.values()]))
  return {'all_tprs': tpr_measures, 'min_tpr': min(tpr_measures.values()),
          'max_tpr': max(tpr_measures.values()), 'avg_tpr': sum(tpr_measures.values())/len(partitions)}


def triangle_participation_ratio(graph, coms):
  cls = nx.triangles(graph, coms)
  nc = [n for n in cls if cls[n] > 0]
  return float(len(nc))/len(coms)


def get_community_modularity(graph, partitions):
  modularities = {}
  try:
      modularities = {'modularity': nx.algorithms.community.modularity(graph, [set(com) 
                                                                               for com in partitions.values()])}
  except:
    return {}
  return modularities

def get_surprise(graph, partitions):
  m = graph.number_of_edges()
  n = graph.number_of_nodes()

  q = 0
  qa = 0
  sp = 0

  for community in partitions.values():
      c = nx.subgraph(graph, community)
      mc = c.number_of_edges()
      nc = c.number_of_nodes()

      q += mc
      qa += scipy.special.comb(nc, 2, exact=True)
  try:
      q = q / m
      qa = qa / scipy.special.comb(n, 2, exact=True)

      sp = m * (q * np.log(q / qa) + (1 - q) * np.log2((1 - q) / (1 - qa)))
  except ZeroDivisionError:
      return {'asymptotic_surprise': np.NAN}
  return {'asymptotic_surprise': sp}

def get_significance(graph, partitions):

  m = graph.number_of_edges()

  binom = scipy.special.comb(m, 2, exact=True)
  p = m / binom

  q = 0

  for community in partitions.values():
      try:
          c = nx.subgraph(graph, community)
          nc = c.number_of_nodes()
          mc = c.number_of_edges()

          binom_c = scipy.special.comb(nc, 2, exact=True)
          pc = mc / binom_c

          q += binom_c * (pc * np.log(pc / p) + (1 - pc) * np.log((1 - pc) / (1 - p)))
      except ZeroDivisionError:
          return {'significance': np.NAN}
  return {'significance': q}


def get_evaluation_metrics(graph, partitions):
  return dict(chain(get_conductance(graph, partitions).items(), 
                    get_triangle_participation_ratio(graph, partitions).items(), 
                    get_community_modularity(graph, partitions).items(), 
                    get_surprise(graph, partitions).items(),
                    get_significance(graph, partitions).items())
             )
  

## Stranger Things

In [35]:
st_graph = load_graph(GRAPHML_FILE)
st_graph = remove_isolated_nodes(st_graph)
st_topic_vec_df = get_topic_frame(TOPIC_VEC_FILE, GRAPH_NODE_FILE)
st_twitter_rank_df = get_topic_frame(TWITTER_RANK_FILE, GRAPH_NODE_FILE)

Name: 
Type: DiGraph
Number of nodes: 1969
Number of edges: 457
Average in degree:   0.2321
Average out degree:   0.2321
Name: 
Type: DiGraph
Number of nodes: 1969
Number of edges: 457
Average in degree:   0.2321
Average out degree:   0.2321

Isolated nodes: 1592

removing isolated nodes...

Name: 
Type: DiGraph
Number of nodes: 377
Number of edges: 457
Average in degree:   1.2122
Average out degree:   1.2122


In [36]:
influential_nodes = get_influential_nodes(st_graph, st_topic_vec_df, st_twitter_rank_df, num_topics=NUM_TOPICS)

In [37]:
initial_communities = step1_assign_initial_communities(st_graph, st_topic_vec_df, influential_nodes)

/Users/soumithri/.local/share/virtualenvs/Thesis-cQCF3LGF/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/soumithri/.local/share/virtualenvs/Thesis-cQCF3LGF/lib/python3.7/site-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [38]:
communities_after_split, community_cosine_sim_df, mapped_nodes = step2_split_community(
    st_topic_vec_df, initial_communities,threshold_percentile_for_split)

Cannot split for community since it has community: [775072732916572160] since it has only one node
Cannot split for community since it has community: [3974634941] since it has only one node
Cannot split for community since it has community: [928458410013675521] since it has only one node
Cannot split for community since it has community: [848606923087908864] since it has only one node
Cannot split for community since it has community: [3228848510] since it has only one node
Cannot split for community since it has community: [2747410528] since it has only one node
Cannot split for community since it has community: [767430013616394240] since it has only one node
Cannot split for community since it has community: [856578556427632643] since it has only one node
Cannot split for community since it has community: [2298700003] since it has only one node
Cannot split for community since it has community: [134787107] since it has only one node
Cannot split for community since it has community: 

In [39]:
communties_after_merge = step3_merge_communities(communities_after_split, community_cosine_sim_df,
                                                 threshold_percentile_for_merge, mapped_nodes)

In [40]:
communties_after_merge

{775072732916572160: [775072732916572160, 767430013616394240],
 3974634941: [3974634941, 885193729098752002],
 928458410013675521: [928458410013675521, 501142275],
 3228848510: [3228848510, 848606923087908864, 2374706366],
 856578556427632643: [856578556427632643, 2338760659],
 134787107: [134787107, 4693956132],
 4090167852: [4090167852, 1654231795],
 327467138: [327467138, 2298700003, 2251175080],
 2982624985: [2982624985, 2381452592],
 2301233508: [2301233508, 3119777010, 197251803],
 93761096: [93761096, 3229506709],
 336872587: [336872587, 345056881, 576840165],
 3607069393: [3607069393, 2980610033],
 2382020768: [2382020768, 742920518353879041],
 751721364: [751721364, 804182695173558272],
 895406294349344768: [895406294349344768, 3007579465, 3033531971, 228891747],
 3033268071: [3033268071, 3382144539, 1268768983],
 2257011015: [2257011015, 813970730],
 2268164780: [2268164780, 726540672, 762213374, 18025029],
 2837077179: [2837077179, 3294535848],
 791138905437462530: [79113890

In [41]:
partitions = make_partitions(st_topic_vec_df, communties_after_merge)
coms_dict = deepcopy(communties_after_merge)

In [42]:
get_evaluation_metrics(st_graph, coms_dict)

{'all_conductances': {775072732916572160: nan,
  3974634941: nan,
  928458410013675521: nan,
  3228848510: nan,
  856578556427632643: nan,
  134787107: nan,
  4090167852: nan,
  327467138: nan,
  2982624985: nan,
  2301233508: nan,
  93761096: nan,
  336872587: nan,
  3607069393: nan,
  2382020768: nan,
  751721364: nan,
  895406294349344768: nan,
  3033268071: nan,
  2257011015: nan,
  2268164780: nan,
  2837077179: nan,
  791138905437462530: nan,
  763875106900672512: nan,
  3302228866: nan,
  711387485700366337: nan,
  780136472615804928: nan,
  2345722098: nan,
  832933001080168450: nan,
  17629149: nan,
  851999658708783104: nan,
  525498287: nan,
  324068936: nan,
  593332602: nan,
  450454763: nan,
  3016439087: nan,
  149406681: nan,
  3939384136: nan,
  910283586: nan,
  511238014: nan},
 'min_conductance': nan,
 'max_conductance': nan,
 'avg_conductance': nan,
 'all_tprs': {775072732916572160: 0.0,
  3974634941: 0.0,
  928458410013675521: 0.0,
  3228848510: 0.0,
  85657855642

In [20]:
type(st_graph)

networkx.classes.digraph.DiGraph

In [21]:
initial_communities

{775072732916572160: [775072732916572160],
 3974634941: [3974634941],
 928458410013675521: [928458410013675521],
 848606923087908864: [848606923087908864],
 3228848510: [3228848510],
 2747410528: [2747410528],
 767430013616394240: [767430013616394240],
 856578556427632643: [856578556427632643],
 2298700003: [2298700003],
 134787107: [134787107],
 4693956132: [4693956132],
 4090167852: [4090167852],
 2338760659: [2338760659],
 345056881: [345056881],
 2251175080: [2251175080],
 327467138: [327467138],
 2287341380: [2287341380],
 2982624985: [2982624985],
 2381452592: [2381452592],
 2852480889: [2852480889]}

In [22]:
influential_nodes

[775072732916572160,
 3974634941,
 928458410013675521,
 848606923087908864,
 3228848510,
 2747410528,
 767430013616394240,
 856578556427632643,
 2298700003,
 134787107,
 4693956132,
 4090167852,
 2338760659,
 345056881,
 2251175080,
 327467138,
 2287341380,
 2982624985,
 2381452592,
 2852480889]

In [23]:
st_topic_vec_df.shape

(377, 11)